### 3.	Створити агреговані таблиці з ключовими фінансовими метриками (Gold Layer).

In [0]:
df_silver = spark.read.format("delta").table("yahoo_data.finance_silver")

Агрегуємо метрики по днях:
- Середню ціну акцій (Avg_Close)
- Максимальну та мінімальну ціну (Max_High, Min_Low)
- Загальний обсяг торгів (Total_Volume)
- Денну зміну ціни (Price_Change)

In [0]:
from pyspark.sql.functions import col, avg, max, min, sum, lag
from pyspark.sql.window import Window

# Групування по днях і компанії
df_gold_daily = df_silver.groupBy("Date", "Ticker").agg(
    avg("Close").alias("Avg_Close"),
    max("High").alias("Max_High"),
    min("Low").alias("Min_Low"),
    sum("Volume").alias("Total_Volume")
)

# Денна зміна ціни для кожної компанії окремо
window_spec = Window.partitionBy("Ticker").orderBy("Date")
df_gold_daily = df_gold_daily.withColumn("Prev_Close", lag("Avg_Close").over(window_spec))
df_gold_daily = df_gold_daily.withColumn("Price_Change", col("Avg_Close") - col("Prev_Close"))

In [0]:
# Перегляд результатів
display(df_gold_daily.head(10))

Date,Ticker,Avg_Close,Max_High,Min_Low,Total_Volume,Prev_Close,Price_Change
2025-01-02,AAPL,243.26319885253906,248.5005651105234,241.23808511721737,55740700,null,null
2025-01-03,AAPL,242.7743682861328,243.5923870862029,241.3079045417172,40244100,243.26319885253906,-0.48883056640625
2025-01-06,AAPL,244.41041564941406,246.73481040856498,242.61474424512696,45045600,242.7743682861328,1.63604736328125
2025-01-07,AAPL,241.62713623046875,244.9590949760797,240.76920518354387,40856000,244.41041564941406,-2.7832794189453125
2025-01-08,AAPL,242.11593627929688,243.12351544341067,239.47231962725974,37628900,241.62713623046875,0.488800048828125
2025-01-10,AAPL,236.280029296875,239.58206138453855,232.4392881886217,61710900,242.11593627929688,-5.835906982421875
2025-01-13,AAPL,233.83592224121094,234.10527676213235,229.16719172961072,49630700,236.280029296875,-2.4441070556640625
2025-01-14,AAPL,232.71861267089844,235.55177458449063,231.9105643637605,39435300,233.83592224121094,-1.1173095703125
2025-01-15,AAPL,237.2975616455078,238.38495013235746,233.86583756026477,39832000,232.71861267089844,4.578948974609375
2025-01-16,AAPL,227.710693359375,237.4372302632713,227.4812511099753,71759100,237.2975616455078,-9.586868286132812


In [0]:
%sql
DROP TABLE yahoo_data.finance_gold;

In [0]:
# збереження в Delta Table
df_gold_daily.write.format("delta").mode("overwrite").saveAsTable("yahoo_data.finance_gold")

In [0]:
%sql
SELECT * FROM yahoo_data.finance_gold LIMIT 5;

Date,Ticker,Avg_Close,Max_High,Min_Low,Total_Volume,Prev_Close,Price_Change
2025-01-02,AAPL,243.26319885253906,248.5005651105234,241.23808511721737,55740700,null,null
2025-01-03,AAPL,242.7743682861328,243.5923870862029,241.3079045417172,40244100,243.26319885253906,-0.48883056640625
2025-01-06,AAPL,244.41041564941406,246.73481040856498,242.61474424512696,45045600,242.7743682861328,1.63604736328125
2025-01-07,AAPL,241.62713623046875,244.9590949760797,240.76920518354387,40856000,244.41041564941406,-2.7832794189453125
2025-01-08,AAPL,242.11593627929688,243.12351544341067,239.47231962725974,37628900,241.62713623046875,0.488800048828125
